# Capstone - PowDay Data Cleaning
Date Started: 2021.10.14<br>
Date Finished: 2021.10.23<br>
William Matthews

### Introduction

The purpose of this notebook is to clean the data obtained from the PowDay app and get it into a form that is ready for EDA.

### Data Set

The data was obtained from a database of push notifications that were sent to the PowDay app.  It includes every update of lift opening, closing, and weather updates.  We are interested only in the lift opening times.  Many thanks to Todd Laney at PowDay who graciously provided access to the data and permission to use it for this capstone project.

### Data Dictionary
`time`    : a datetime stamp indiciating when the notification was sent to the app.  This will be used as a proxy for actual lift opening times.<br>
`cmd`     : a string indicating that a PUSH to the app happened.<br>
`message` : a string including what updates were sent to the app.<br>
`status`  : an integer representing some internal status for the PowDay app.

### Import Libraries and Data

In [1]:
# working with dataFrames
import pandas as pd

# for creating datetime objects for comparisons
import datetime

In [2]:
pow_df = pd.read_csv('Data/PowDayHistory.csv')

On the first attept to load the PowDay data there was an error at line 69,884 of the text file.  The parser found more than the four fields expected.  A quick visual inspection of the data via a text editor revealed three rows (69,884 through 69,886) that had additional commas in the `message` field.  A few edits to those lines in the text editor to remove the additional comma had no effect on the data.

Second attempt we received one more error of the same type at lin 70,936.  Visual inspection showed two more at 70,942 and 70,949, all which were corrected.

Third pass, 4 more issues at 71,810, 71,817, 71,824, 71,831.  Almost all of the issues have the pattern 'snow, occasionally ...', so the search function in the text editor was used to locate and fix all of the issues with this pattern (26 more occurances of this issue).  Interestinglly, some of these were at line numbers before the spot the parser had made it to.  Continuing on:
- Fourth pass, another errent pattern was detected ', then sun'.  Three occurances of this were found and corrected.
- Fifth pass, ', clearing in the evening ...'.  Three occurances found and corrected.
- Sixth pass, ', easing to isolated ...'.  Six occrances found and corrected.
- Seventh pass, ', easing to scattered ...'.  Three occrances found and corrected.
- Eigth pass, ', increasing to ...'. Three occurances found and corrected
- Nineth pass, ', becoming a mix ...'. Three occurances found and corrected.
- Tenth pass, ', ending near ...'. Three occurances found and corrected.
- Eleventh pass, ', then partial ...'. Three occurances found and corrected.

Twelfth pass was successful!  Less than 30 minutes of work.  Not bad!

### Data Cleaning

In [3]:
# quick look at the number of records
pow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89671 entries, 0 to 89670
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   time     89671 non-null  object
 1   cmd      89671 non-null  object
 2   message  89573 non-null  object
 3   status   89671 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 2.7+ MB


From the above we can see there are a very small number of rows with any information missing.  Since the `message` field is one of the two fields we are most interested in, let's take a look at the 98 records to see if there is anything worth keeping there.

In [4]:
# get index of null rows
pow_df[pow_df['message'].isna() == True]

,time,cmd,message,status
24044,2016-11-28 10:20:05,PUSH,NaN,94
24045,2016-11-28 10:22:04,PUSH,NaN,73
24046,2016-11-28 10:30:10,PUSH,NaN,187
24047,2016-11-28 10:30:21,PUSH,NaN,860
24048,2016-11-28 11:00:05,PUSH,NaN,94
...,...,...,...,...
24137,2016-11-30 10:55:04,PUSH,NaN,71
24138,2016-11-30 11:00:05,PUSH,NaN,94
24139,2016-11-30 11:27:04,PUSH,NaN,98
24140,2016-11-30 11:49:14,PUSH,NaN,985


It looks like there is nothing of interest there.  Let's go ahead and drop all our null values.

In [5]:
# confrim it is going to do what we want
pow_df.dropna(axis = 0,
              how = 'any').shape[0] == (pow_df.shape[0] - 98)

True

In [6]:
# drop and copy
pow_df2 = pow_df.dropna(axis = 0,
                        how = 'any').copy()

#confirm
pow_df2.isna().sum()

time       0
cmd        0
message    0
status     0
dtype: int64

In [7]:
# another look at total records
pow_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89573 entries, 0 to 89670
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   time     89573 non-null  object
 1   cmd      89573 non-null  object
 2   message  89573 non-null  object
 3   status   89573 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 3.4+ MB


In [8]:
pow_df2.nunique()

time       88706
cmd            1
message     5159
status      2176
dtype: int64

So based on the above, there are a few things to do:
- change `time` into a datetime format
- drop the `cmd` column as it provides no info
- drop the `status` column as there is no information in the numbers.
- restrict our information to the ski seasons only within our data range as per [snowpak.ca](https://www.snowpak.ca/british-columbia/whistler-blackcomb/opening-dates)

In [9]:
# comparison for to datetime to make sure it is doing what we want
pow_df2['time']

0        2014-12-31 15:18:02
1        2014-12-31 15:18:13
2        2014-12-31 15:30:03
3        2014-12-31 15:30:14
4        2014-12-31 15:31:02
                ...         
89666    2021-05-31 17:01:03
89667    2021-05-31 17:01:05
89668    2021-05-31 20:00:03
89669    2021-05-31 20:01:02
89670    2021-05-31 20:01:19
Name: time, Length: 89573, dtype: object

In [10]:
# time to datetime check
pd.to_datetime(pow_df2['time'],
               infer_datetime_format = True)

0       2014-12-31 15:18:02
1       2014-12-31 15:18:13
2       2014-12-31 15:30:03
3       2014-12-31 15:30:14
4       2014-12-31 15:31:02
                ...        
89666   2021-05-31 17:01:03
89667   2021-05-31 17:01:05
89668   2021-05-31 20:00:03
89669   2021-05-31 20:01:02
89670   2021-05-31 20:01:19
Name: time, Length: 89573, dtype: datetime64[ns]

In [11]:
# move ahead
pow_df2['time'] = pd.to_datetime(pow_df2['time'],
                  infer_datetime_format = True)

In [12]:
# quick check
pow_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89573 entries, 0 to 89670
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   time     89573 non-null  datetime64[ns]
 1   cmd      89573 non-null  object        
 2   message  89573 non-null  object        
 3   status   89573 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 3.4+ MB


In [13]:
# drop cmd and status columns
pow_df3 = pow_df2.drop(labels = ['cmd', 'status'],
             axis = 1).copy()

In [14]:
# confirm it worked
pow_df3

,time,message
0,2014-12-31 15:18:02,Showcase is CLOSED
1,2014-12-31 15:18:13,Blackcomb Glacier is CLOSED
2,2014-12-31 15:30:03,Symphony is CLOSED
3,2014-12-31 15:30:14,Flute Bowl is CLOSED
4,2014-12-31 15:31:02,7th Heaven is CLOSED
...,...,...
89666,2021-05-31 17:01:03,Blackcomb Gondola is CLOSED
89667,2021-05-31 17:01:05,Peak 2 Peak is CLOSED
89668,2021-05-31 20:00:03,Cloudy with clear periods.\nWinds: Calm\nTempe...
89669,2021-05-31 20:01:02,Fitzsimmons is CLOSED


#### Data Cleaning - Filtering For Ski Seasons

<a id='key_decision_2'></a>
Before we can filter our data set for dates in the ski season, we need to generate a list of all the opening and closing dates.  Dates were taken from [snowpak.ca](https://www.snowpak.ca/british-columbia/whistler-blackcomb/opening-dates) as a base line.

On review of these dates, it looks like many of the seasons extend well into May.  This is not when most people are skiing.  Also, Whistler/Blackcomb usually shuts at least one of the mountains by the end of April at the latest.  If one of the mountains has all the lifts closed due to different mountain closing dates, that is not something we are trying to predict.  Based on the goal of the prediction algorithem being to help people plan their day on the mountain, we are going to restrict the model to working on the 'core season'.  Whistler/Blackcomb qualifies core season as [December 7 - April 17](https://www.whistlerblackcomb.com/info/epic-coverage.aspx) for the 2021/2022 season.

We are going to set our core season end dates to match Whistler/Blackcomb's. For the core season start date, we are going to keep the start dates of each season.  We intened to use snow pack data as predictors, and believe this will be a good driver for when lifts will open in the season (ex. Peak chair might not open until the snow base is > 150cm).  With that in mind, we will define our core season as the opening date provided by snowpak.ca until April 17th of each year.

The only exception to the opening dates is the opening of the 2014 season.  Because the earliest record of lift openings is 2014-12-31, we will use that in place of the opening date provided by snowpak.ca. The exceptions for ending dates are in the seasons of 2019/2020 and 2020/2021.  Both of these seasons terminated in March due to COVID 19.  If it turns out that these short seasons (late start or early close) are causing havoc with model we will consider either imputing an average number of open/closed days for the missing portions of the season, or shortening all seasons to a smaller core of January 1st through March 14th of each year. 

On a technical note, the datetime object automatically defaults the time to `00:00:00`, which works for the opening day, but we will have to specify `23:59:59` for the closing day to ensure all lift opening times on the last day are captured.

In [15]:
# list to store tuples of (open, close)
open_close = []

# add all open close dates as per snowpak.ca + exception
open_close.append((datetime.datetime(2014, 12, 31), datetime.datetime(2015, 4, 17, 23, 59, 59)))
open_close.append((datetime.datetime(2015, 11, 19), datetime.datetime(2016, 4, 17, 23, 59, 59)))
open_close.append((datetime.datetime(2016, 11, 23), datetime.datetime(2017, 4, 17, 23, 59, 59)))
open_close.append((datetime.datetime(2017, 11, 17), datetime.datetime(2018, 4, 17, 23, 59, 59)))
open_close.append((datetime.datetime(2018, 11, 22), datetime.datetime(2019, 4, 17, 23, 59, 59)))
open_close.append((datetime.datetime(2019, 11, 26), datetime.datetime(2020, 3, 14, 23, 59, 59)))
open_close.append((datetime.datetime(2020, 11, 26), datetime.datetime(2021, 3, 29, 23, 59, 59)))

In [16]:
# restrict data to ski seasons
def season_limit(d, open_close):
    """
    Takes a datetime object and determines if it falls into one of our ski-season time windows as defined below
    _________________
    
    Parameters:
                d: a datetime object
                open_close: a list of 2-tuples storing datetime objects in the form (opening day, closing day )
    ________________
    
    Returns:
            True or False depnding if the data falls into one of the open_close ranges defined below
    
    """
        
    # loop through all.  return True if there is any match.
    for opened, closed in open_close:
        if d >= opened and d <= closed:
            return True
        
    # default condition if whole loop traversed (our date is not in one of our ski seasons)
    return False
    

In [17]:
# filter for ski seasons only -first check
pow_df3[pow_df3['time'].apply(season_limit, args = (open_close,)) == True]['time'].describe()

/tmp/ipykernel_6463/235975019.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  pow_df3[pow_df3['time'].apply(season_limit, args = (open_close,)) == True]['time'].describe()


count                   65725
unique                  65663
top       2017-03-30 17:37:04
freq                        4
first     2014-12-31 15:18:02
last      2021-03-29 15:06:02
Name: time, dtype: object

In [18]:
# filter for ski seasons only
pow_df4 = pow_df3[pow_df3['time'].apply(season_limit, args = (open_close,)) == True]

# second check with 2016 shoulder season (last day of 2015/2016 through first day of 2016/20217 inclusive).
# Expecting records from two days only.
pow_df4[(pow_df4['time'] <= datetime.datetime(2016, 11, 23, 23, 59, 59)) & 
        (pow_df4['time'] >= datetime.datetime(2016, 4, 17))]['time'].apply(lambda d: d.day).unique()

array([17, 23])

Looks like that has worked and we have dates for our ski seasons only.  Let's move on to see how many records of openings we have.

In [19]:
# check to see how many chair lift OPEN messages me have.  These will be the target values for our training model
pow_df4[pow_df4['message'].str.contains('OPEN')].count()

time       25155
message    25155
dtype: int64

So it looks like we have just over 25,000 records containing the word open.  That is a great start, but before we go any further we need to layout a plan and stick to it.  Specifically we need to check for duplicate rows before we do anything else.  This will take some work as we will have to group records by time windows to make sure there are not any multiple 'OPEN' messages the same day for the same chair.  So we need to parse our info into chair columns first.

And before we do that, we will pull all of the 'OPEN' records into a new data frame and all the remaining records into another new data frame.  This will make searching the non-'OPEN' data for anything important a little easier.  

In [20]:
# df to store lift information
lift_df = pow_df4[pow_df4['message'].str.contains('OPEN')]

# df to store other information
working_df = pow_df4[pow_df4['message'].str.contains('OPEN') != True]


In [21]:
# confirm it worked
display(lift_df)
display(working_df)

,time,message
29,2014-12-31 16:05:03,Emerald is OPEN
36,2015-01-01 09:01:04,Excalibur Gondola is OPEN
37,2015-01-01 09:01:14,Excelerator is OPEN
38,2015-01-01 09:01:24,Solar Coaster is OPEN
39,2015-01-01 09:01:35,Wizard is OPEN
...,...,...
89518,2021-03-29 10:09:37,Harmony Zone is OPEN
89519,2021-03-29 10:14:06,Excalibur Gondola is OPEN
89520,2021-03-29 10:30:08,Symphony is OPEN
89521,2021-03-29 10:30:14,Flute Bowl is OPEN


,time,message
0,2014-12-31 15:18:02,Showcase is CLOSED
1,2014-12-31 15:18:13,Blackcomb Glacier is CLOSED
2,2014-12-31 15:30:03,Symphony is CLOSED
3,2014-12-31 15:30:14,Flute Bowl is CLOSED
4,2014-12-31 15:31:02,7th Heaven is CLOSED
...,...,...
89552,2021-03-29 15:04:34,Excelerator is CLOSED
89553,2021-03-29 15:04:49,Creekside Gondola is CLOSED
89554,2021-03-29 15:05:04,Emerald is CLOSED
89555,2021-03-29 15:05:42,BADGE=0


In [22]:
# one more check
(lift_df.shape[0] + working_df.shape[0]) == pow_df4.shape[0]

True

#### Data Cleaning - Extracting Chair Names

Great.  Looks like we have successfully serperated our data.  Let's focus on the `lift_df` for now.  Current to-do list:
- Parse out each chair name into a new column `chair`
- Check for any duplicate data, defined as a given chair opening multiple times in one day
- Add `day_status` column and fill with 1's (True) to represent open.
- Check for any 'missing data'.  This would be a day during the ski season for which we have no record for a given lift.  Add a row for that lift for that day, and impute 0 into the `day_status` column to indicate the chair never opened that day.
- Drop `message` column

In [23]:
# confirm string slice does what we want
lift_df['message'].apply(lambda s: s[:-8])

29                 Emerald
36       Excalibur Gondola
37             Excelerator
38           Solar Coaster
39                  Wizard
               ...        
89518         Harmony Zone
89519    Excalibur Gondola
89520             Symphony
89521           Flute Bowl
89522        Symphony Bowl
Name: message, Length: 25155, dtype: object

In [24]:
# use string slicing to grab everything before ' is OPEN'
lift_df['lift'] = lift_df['message'].apply(lambda s: s[:-8])

/tmp/ipykernel_6463/3377178295.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lift_df['lift'] = lift_df['message'].apply(lambda s: s[:-8])


In [25]:
# check it worked
lift_df

,time,message,lift
29,2014-12-31 16:05:03,Emerald is OPEN,Emerald
36,2015-01-01 09:01:04,Excalibur Gondola is OPEN,Excalibur Gondola
37,2015-01-01 09:01:14,Excelerator is OPEN,Excelerator
38,2015-01-01 09:01:24,Solar Coaster is OPEN,Solar Coaster
39,2015-01-01 09:01:35,Wizard is OPEN,Wizard
...,...,...,...
89518,2021-03-29 10:09:37,Harmony Zone is OPEN,Harmony Zone
89519,2021-03-29 10:14:06,Excalibur Gondola is OPEN,Excalibur Gondola
89520,2021-03-29 10:30:08,Symphony is OPEN,Symphony
89521,2021-03-29 10:30:14,Flute Bowl is OPEN,Flute Bowl


In [26]:
# further check
lift_df['lift'].unique()

array(['Emerald', 'Excalibur Gondola', 'Excelerator', 'Solar Coaster',
       'Wizard', 'Red Chair', 'Creekside Gondola', 'Fitzsimmons',
       'Olympic', 'Village Gondola', 'Garbanzo', 'Jersey Cream',
       'Crystal', '7th Heaven', 'Glacier', 'Peak 2 Peak', 'Harmony',
       'Peak', 'Glacier Bowl', 'Harmony Zone', 'West Bowl',
       'Whistler Bowl', "Xhiggy's Meadow", "Spanky's Ladder",
       'Catskinner', 'T-BAR', 'Horstman', 'Showcase', 'Blackcomb Glacier',
       "Franz's", 'Symphony', 'Flute Bowl', 'Whistler Alpine',
       'Blackcomb Alpine', 'Symphony Bowl', 'Whistler Village Gondola',
       'Blackcomb Gondola', 'Blackcomb Gondola (Lower)',
       'Excalibur Gondola (Lower)', 'Whistler Village Gondola (Lower)'],
      dtype=object)

<a id='key_decision_5'></a>
From the above, we can see that there are a few items in here that don't make sense for various reasons.  Some are ski runs or areas as opposed to ski lifts, so we are going to remove those.  That said, there are a few areas that we are going to keep.  `Spanky's Ladder`, `Blackcomb Glacier`, and `Flute Bowl` are areas that may be closed while all lifts are open.  Their status is a good indicator of current avalanche conditions and might be useful in predicting if/when lifts will open. 

Also worth noting, `Solar Coaster` and `Wizard` chairs were both removed and replaced by `Blackcomb Gondola` in 2018, so they are not going to be much use going forward.  That said, we believe the opening of the upper lifts may be well predicted by the opening times of the lower lifts, specifically on edge case days.  To address this, we will replace all `Solar Coaster` openings with `Blackcomb Gondola` and all `Wizard` openings with `Blackcomb Gondola (Lower)` as this corresponds to how the gondola replaced the two chairlifts.

In [27]:
# removing non-ski lifts
to_remove = ['Glacier Bowl',
             'Harmony Zone',
             'West Bowl',
             'Whistler Bowl',
             "Xhiggy's Meadow",
             'Whistler Alpine',
             'Blackcomb Alpine',
             'Symphony Bowl'
             ]
# filtering out the lifts to remove
lift_df['lift'].apply(lambda lift: lift not in to_remove)

29        True
36        True
37        True
38        True
39        True
         ...  
89518    False
89519     True
89520     True
89521     True
89522    False
Name: lift, Length: 25155, dtype: bool

In [28]:
# check the above makes works
lift_df

,time,message,lift
29,2014-12-31 16:05:03,Emerald is OPEN,Emerald
36,2015-01-01 09:01:04,Excalibur Gondola is OPEN,Excalibur Gondola
37,2015-01-01 09:01:14,Excelerator is OPEN,Excelerator
38,2015-01-01 09:01:24,Solar Coaster is OPEN,Solar Coaster
39,2015-01-01 09:01:35,Wizard is OPEN,Wizard
...,...,...,...
89518,2021-03-29 10:09:37,Harmony Zone is OPEN,Harmony Zone
89519,2021-03-29 10:14:06,Excalibur Gondola is OPEN,Excalibur Gondola
89520,2021-03-29 10:30:08,Symphony is OPEN,Symphony
89521,2021-03-29 10:30:14,Flute Bowl is OPEN,Flute Bowl


In [29]:
# filtering out non-ski lifts
lift_df2 = lift_df[lift_df['lift'].apply(lambda lift: lift not in to_remove)]

# check
for non_lift in to_remove:
    if non_lift in lift_df2['lift'].unique():
        print('Did not work right!')

Looks like the above has worked.  Let's move onto replacing `Wizard` and `Solar Coaster`.

In [30]:
def lift_replace(lift):
    """
    Replace Wizard and Solar Coaster with the appropriate Blackcomb Gondola section
    """
    if lift == 'Wizard':
        return 'Blackcomb Gondola (Lower)'
    elif lift == 'Solar Coaster':
        return 'Blackcomb Gondola'
    else:
        return lift

# confirm it will work
lift_df2['lift'].apply(lift_replace).unique()

array(['Emerald', 'Excalibur Gondola', 'Excelerator', 'Blackcomb Gondola',
       'Blackcomb Gondola (Lower)', 'Red Chair', 'Creekside Gondola',
       'Fitzsimmons', 'Olympic', 'Village Gondola', 'Garbanzo',
       'Jersey Cream', 'Crystal', '7th Heaven', 'Glacier', 'Peak 2 Peak',
       'Harmony', 'Peak', "Spanky's Ladder", 'Catskinner', 'T-BAR',
       'Horstman', 'Showcase', 'Blackcomb Glacier', "Franz's", 'Symphony',
       'Flute Bowl', 'Whistler Village Gondola',
       'Excalibur Gondola (Lower)', 'Whistler Village Gondola (Lower)'],
      dtype=object)

In [31]:
# replace lifts as above
lift_df2['lift'] = lift_df2['lift'].apply(lift_replace)

# triple check!
lift_df2['lift'].unique()

/tmp/ipykernel_6463/2491920004.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lift_df2['lift'] = lift_df2['lift'].apply(lift_replace)


array(['Emerald', 'Excalibur Gondola', 'Excelerator', 'Blackcomb Gondola',
       'Blackcomb Gondola (Lower)', 'Red Chair', 'Creekside Gondola',
       'Fitzsimmons', 'Olympic', 'Village Gondola', 'Garbanzo',
       'Jersey Cream', 'Crystal', '7th Heaven', 'Glacier', 'Peak 2 Peak',
       'Harmony', 'Peak', "Spanky's Ladder", 'Catskinner', 'T-BAR',
       'Horstman', 'Showcase', 'Blackcomb Glacier', "Franz's", 'Symphony',
       'Flute Bowl', 'Whistler Village Gondola',
       'Excalibur Gondola (Lower)', 'Whistler Village Gondola (Lower)'],
      dtype=object)

#### Data Cleaning - Multiple 'Opens' per Day

The above warning does not seem to have affected the outcome, so we will move on with the remainder of our list:
- Check for any duplicate data, defined as a given chair opening multiple times in one day
- Add `day_status` column and fill with 1's (True) to represent open.
- Check for any 'missing data'.  This would be a day during the ski season for which we have no record for a given lift.  Add a row for that lift for that day, and impute 0 into the `day_status` column to indicate the chair never opened that day.
- Drop `message` column

In order to leverage pandas duplicated/drop_duplicates function we are going to temporarily create `Year`, `Month`, and `Day` columns.  We will then check for duplicates on this subset of columns plus `lift`.

In [32]:
# create year
lift_df2['year'] = lift_df2['time'].apply(lambda d: d.year)

# create month
lift_df2['month'] = lift_df2['time'].apply(lambda d: d.month)

# create day
lift_df2['day'] = lift_df2['time'].apply(lambda d: d.day)

#check it worked
lift_df2

/tmp/ipykernel_6463/2821440699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lift_df2['year'] = lift_df2['time'].apply(lambda d: d.year)
/tmp/ipykernel_6463/2821440699.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lift_df2['month'] = lift_df2['time'].apply(lambda d: d.month)
/tmp/ipykernel_6463/2821440699.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,time,message,lift,year,month,day
29,2014-12-31 16:05:03,Emerald is OPEN,Emerald,2014,12,31
36,2015-01-01 09:01:04,Excalibur Gondola is OPEN,Excalibur Gondola,2015,1,1
37,2015-01-01 09:01:14,Excelerator is OPEN,Excelerator,2015,1,1
38,2015-01-01 09:01:24,Solar Coaster is OPEN,Blackcomb Gondola,2015,1,1
39,2015-01-01 09:01:35,Wizard is OPEN,Blackcomb Gondola (Lower),2015,1,1
...,...,...,...,...,...,...
89511,2021-03-29 09:28:27,Peak is OPEN,Peak,2021,3,29
89517,2021-03-29 10:09:30,Harmony is OPEN,Harmony,2021,3,29
89519,2021-03-29 10:14:06,Excalibur Gondola is OPEN,Excalibur Gondola,2021,3,29
89520,2021-03-29 10:30:08,Symphony is OPEN,Symphony,2021,3,29


In [33]:
# check for duplicates
dups = lift_df2.duplicated(subset = ['year', 'month', 'day', 'lift']).sum()
dups

615

In [34]:
# check for duplicates
lift_df2.duplicated(subset = ['year', 'month', 'day', 'lift'], keep = False).sum()

1200

In [35]:
# dups as a % of rows
dups/lift_df2.shape[0]

0.030255325429232056

From the above output it looks like we have 640 duplicated records representing just under 3% of our data.  Let's get a look at these records to make sure they are true duplicates.

In [36]:
# get all duplicates
dups_lift_df2 = lift_df2[lift_df2.duplicated(subset = ['year', 'month', 'day', 'lift'], keep = False) == True]

# take a look
display(dups_lift_df2.sort_values(['year', 'month', 'day', 'lift']).head(50))
display(dups_lift_df2.sort_values(['year', 'month', 'day', 'lift']).tail(50))

,time,message,lift,year,month,day
50,2015-01-01 10:08:09,7th Heaven is OPEN,7th Heaven,2015,1,1
51,2015-01-01 10:08:27,7th Heaven is OPEN,7th Heaven,2015,1,1
53,2015-01-01 10:09:12,7th Heaven is OPEN,7th Heaven,2015,1,1
55,2015-01-01 10:10:05,7th Heaven is OPEN,7th Heaven,2015,1,1
52,2015-01-01 10:08:39,Glacier is OPEN,Glacier,2015,1,1
54,2015-01-01 10:09:23,Glacier is OPEN,Glacier,2015,1,1
56,2015-01-01 10:10:15,Glacier is OPEN,Glacier,2015,1,1
43,2015-01-01 09:03:33,Olympic is OPEN,Olympic,2015,1,1
68,2015-01-01 10:21:05,Olympic is OPEN,Olympic,2015,1,1
112,2015-01-02 09:01:30,Red Chair is OPEN,Red Chair,2015,1,2


,time,message,lift,year,month,day
85100,2021-02-02 11:37:41,Excalibur Gondola is OPEN,Excalibur Gondola,2021,2,2
85412,2021-02-06 09:03:21,Excalibur Gondola is OPEN,Excalibur Gondola,2021,2,6
85442,2021-02-06 11:58:06,Excalibur Gondola is OPEN,Excalibur Gondola,2021,2,6
85413,2021-02-06 09:03:36,Excelerator is OPEN,Excelerator,2021,2,6
85444,2021-02-06 12:40:06,Excelerator is OPEN,Excelerator,2021,2,6
85426,2021-02-06 10:37:07,Jersey Cream is OPEN,Jersey Cream,2021,2,6
85441,2021-02-06 11:40:10,Jersey Cream is OPEN,Jersey Cream,2021,2,6
85492,2021-02-07 09:01:36,Excalibur Gondola is OPEN,Excalibur Gondola,2021,2,7
85523,2021-02-07 11:15:06,Excalibur Gondola is OPEN,Excalibur Gondola,2021,2,7
86041,2021-02-14 09:04:08,Whistler Village Gondola is OPEN,Whistler Village Gondola,2021,2,14


<a id='key_decision_1'></a>
Based on our knowledge of Whistler/Blackcomb operations, it looks like most of these are true duplicates.  Even though the time interval on some of these is large and might be realistic, it is very rare for a chair to close and re-open in the same day.  Also, a lot of the second opening times are past 14:00 hrs, and we have never seen a chair on the mountain open at that time.  Typical operations have the mountain beginning to shut down by 15:00 - 15:30 each day.

We are going to go ahead and drop all of the duplicates indentified and then remove the `year`, `month`, and `day` columns that were temporarily added.  This means we are taking the earliest opening for any lift on any given day.

That said, this may end up being something that affects the model quite significantly, especially if we try to predict opening times with any time window that would be useful to skiers (ex. +/- 20 minutes).  If the model is struggling we will consider returing here and re-evaluating our approach.

In [37]:
# remove duplicates and copy
lift_df3 = lift_df2.drop_duplicates(subset = ['year', 'month', 'day', 'lift'])

# check it worked
lift_df3.shape[0] == (lift_df2.shape[0] - dups.sum())

True

In [38]:
# drop year/month/day
display(lift_df3.shape)

lift_df3 = lift_df3.drop(labels = ['year', 'month', 'day'], axis = 1)

# check shapes before and after differ by 3 columns only
lift_df3.shape

(19712, 6)

(19712, 3)

In [39]:
# visual check
lift_df3

,time,message,lift
29,2014-12-31 16:05:03,Emerald is OPEN,Emerald
36,2015-01-01 09:01:04,Excalibur Gondola is OPEN,Excalibur Gondola
37,2015-01-01 09:01:14,Excelerator is OPEN,Excelerator
38,2015-01-01 09:01:24,Solar Coaster is OPEN,Blackcomb Gondola
39,2015-01-01 09:01:35,Wizard is OPEN,Blackcomb Gondola (Lower)
...,...,...,...
89511,2021-03-29 09:28:27,Peak is OPEN,Peak
89517,2021-03-29 10:09:30,Harmony is OPEN,Harmony
89519,2021-03-29 10:14:06,Excalibur Gondola is OPEN,Excalibur Gondola
89520,2021-03-29 10:30:08,Symphony is OPEN,Symphony


With the temporary columns removed, we can move on with our current to-do list:
- Add `day_status` column and fill with 1's (True) to represent open.
- Check for any 'missing data'.  This would be a day during the ski season for which we have no record for a given lift.  Add a row for that lift for that day, and impute 0 into the `day_status` column to indicate the chair never opened that day.
- Drop `message` column

In [40]:
# add day_status column
lift_df3['day_status'] = 1

# confirm it worked
lift_df3

,time,message,lift,day_status
29,2014-12-31 16:05:03,Emerald is OPEN,Emerald,1
36,2015-01-01 09:01:04,Excalibur Gondola is OPEN,Excalibur Gondola,1
37,2015-01-01 09:01:14,Excelerator is OPEN,Excelerator,1
38,2015-01-01 09:01:24,Solar Coaster is OPEN,Blackcomb Gondola,1
39,2015-01-01 09:01:35,Wizard is OPEN,Blackcomb Gondola (Lower),1
...,...,...,...,...
89511,2021-03-29 09:28:27,Peak is OPEN,Peak,1
89517,2021-03-29 10:09:30,Harmony is OPEN,Harmony,1
89519,2021-03-29 10:14:06,Excalibur Gondola is OPEN,Excalibur Gondola,1
89520,2021-03-29 10:30:08,Symphony is OPEN,Symphony,1


In [41]:
# drop message columns
lift_df3 = lift_df3.drop('message', axis = 1)

# confirm it worked
lift_df3

,time,lift,day_status
29,2014-12-31 16:05:03,Emerald,1
36,2015-01-01 09:01:04,Excalibur Gondola,1
37,2015-01-01 09:01:14,Excelerator,1
38,2015-01-01 09:01:24,Blackcomb Gondola,1
39,2015-01-01 09:01:35,Blackcomb Gondola (Lower),1
...,...,...,...
89511,2021-03-29 09:28:27,Peak,1
89517,2021-03-29 10:09:30,Harmony,1
89519,2021-03-29 10:14:06,Excalibur Gondola,1
89520,2021-03-29 10:30:08,Symphony,1


#### Data Cleaning - Imputing Missing Days and Addressing Lift Nomenclature

The final step in preparing the lift data is to ensure there is a record for each day of every season.  Because we have all the days lifts opened, it will be assumed these lifts did not open on the days that are missing.  We will generate records for each missing day an mark them as closed with a 0 in the `day_status` column.

Before we proceed with imputing/creating records, let's check to see how many records we are going to be imputing.  We will build this as a function so we can use it to check our records after imputing to make sure the imputation worked.

_Personal Note: Another 0.75 hrs to this point_

In [60]:
def missing_lift_days(df, open_close):
    """
    Takes a current data frame of lift records and a list of season dates. Returns how many
    days are missing for each chair lift in each season.
    _________________
    
    Parameters:
                df: pandas dataFrame object, columns[time, lift, day_status]
                open_close: a list of 2-tuples storing datetime objects in the form (opening day, closing day)
    ________________
    
    Returns:
           pandas dataframe object, with lifts as rows, seasons as columns, and missing days as values 
    
    """    
    # dict to store {lift: dict{season: days_missed}
    lift_dict = {}
    
    # for each chair
    for lift in df['lift'].unique():
        
        # dict to store {season: days_missed}
        season_dict = {}
        
        # for each season - utilize season dates stored earlier
        for opened, closed in open_close:

            # extract days from time delta object
            days_in_season = (closed - opened).days + 1

            # extract number of records
            season_records = df[(df['lift'] == lift) & 
                                (df['time'] >= opened) & 
                                (df['time'] <= closed)].shape[0]

            # calculate days with out records in the season and store
            missing_days = days_in_season - season_records
            season_label = f"{opened.year}/{closed.year}"
            season_dict[season_label] = missing_days
            #print(f"{lift} in {opened.year}/{closed.year} missing {days_in_season - season_records} days")
        lift_dict[lift] = season_dict
    
    # create and return dataFrame
    df = pd.DataFrame(data = lift_dict).T
    return df

In [43]:
# print days per season for reference
for opened, closed in open_close:
    season_days = (closed - opened).days + 1
    print(f"{opened.date()} / {closed.date()}: {season_days}")

2014-12-31 / 2015-04-17: 108
2015-11-19 / 2016-04-17: 151
2016-11-23 / 2017-04-17: 146
2017-11-17 / 2018-04-17: 152
2018-11-22 / 2019-04-17: 147
2019-11-26 / 2020-03-14: 110
2020-11-26 / 2021-03-29: 124


In [44]:
# generate missing days
display(missing_lift_days(lift_df3, open_close))

,2014/2015,2015/2016,2016/2017,2017/2018,2018/2019,2019/2020,2020/2021
Emerald,6,11,3,0,11,3,1
Excalibur Gondola,8,23,6,6,11,2,5
Excelerator,8,22,6,6,11,2,3
Blackcomb Gondola,8,22,5,6,36,3,3
Blackcomb Gondola (Lower),9,22,6,6,147,107,117
Red Chair,7,10,3,0,12,3,0
Creekside Gondola,9,12,4,0,13,3,1
Fitzsimmons,20,47,23,25,32,24,0
Olympic,7,19,8,6,32,24,19
Village Gondola,8,12,3,0,147,110,124


From the above output, we can see there are many lifts with reasonable amounts of missing days in a given season when compared to the total days in a season.  This is based on our personal experience having skied Whistler/Blackcomb for the past 10 seasons averaging of 35 days a season.  How ever, there are a number of issues:
- Blackcomb Gondola (Lower) for 2018-2021
- Village Gondola for 2018-2021
- Horstman for 2017-2021
- Franz's chair for most seasons
- Whistler Village Gondola 2014-2018
- Excalibur Gondola (Lower) all seasons
- Whitler Village Gondola (Lower) all seasons.

Let's address these one at a time with the easiest first.  Franz's chair is a slow triple chair that services a subset of the terrain serviced by the Red Chair.  In our experience on the mountain, it is rarely open and the above numbers confirm that.  We believe the Franz's chair numbers are an accurate reflection of reality and are going to leave them as is.

Second, the Village Gondola and Whistler Village Gondola are the same lift.  It looks like the name of the lift was changed in the data set bewteen the 2017/2018 and the 2018/2019 seasons.  We will solve this by amalgamating the data from the 'two lifts' into one lift.  We will keep the seasons with the lowest number of missed days from each.  The Whistler Village Gondola is the primary access route up Whistler Mountain from Whistler Village and it is rare to see it closed, so this seems the most sensible approach.

In [45]:
# confirm there are no Whistler Village Gondola records up to end of 2017/2018 season
print('Confirm no WVG records:', end = ' ')

display(lift_df3[(lift_df3['lift'] == 'Whistler Village Gondola') &
         (lift_df3['time'] <= open_close[3][1])].shape)

# get index of every Village Gondola record up to end of 2017/2018 season
village_gondy_indicies = lift_df3[(lift_df3['lift'] == 'Village Gondola') & 
                                  (lift_df3['time'] <= open_close[3][1])].index

# for each of these records, change the lift to Whistler Village Gondola
for i in village_gondy_indicies:
    lift_df3.loc[i, 'lift'] = 'Whistler Village Gondola'
    
# confirm no remaining records of Village Gondola
print('Confirm no VG records remain:')

display(lift_df3[lift_df3['lift'] == 'Village Gondola'].shape)

# final check it worked!
display(missing_lift_days(lift_df3, open_close))

Confirm no WVG records: 

(0, 3)

Confirm no VG records remain:


(0, 3)

,2014/2015,2015/2016,2016/2017,2017/2018,2018/2019,2019/2020,2020/2021
Emerald,6,11,3,0,11,3,1
Excalibur Gondola,8,23,6,6,11,2,5
Excelerator,8,22,6,6,11,2,3
Blackcomb Gondola,8,22,5,6,36,3,3
Blackcomb Gondola (Lower),9,22,6,6,147,107,117
Red Chair,7,10,3,0,12,3,0
Creekside Gondola,9,12,4,0,13,3,1
Fitzsimmons,20,47,23,25,32,24,0
Olympic,7,19,8,6,32,24,19
Whistler Village Gondola,8,12,3,0,12,4,4


The above has worked and we have solved our second issue.  Next is the Horstman T-Bar.  This is a bit of a sobering reality for skiers in general as it has been [removed due to climate change](https://unofficialnetworks.com/2020/07/13/whistler-removes-ski-lift-climate-change/).  As this lift is no longer in operation, and no lift has replaced it, we will remove it from the data set.  No use predicting if a non-existent lift will open!

In [46]:
# get indicies of horstman t-bar
horstman_indicies = lift_df3[lift_df3['lift'] == 'Horstman'].index

# drop all Horstman t-bar rows
lift_df4 = lift_df3.drop(index = horstman_indicies).copy()

# confirm it worked
('Horstman' in lift_df4['lift'].unique())

False

The above has worked well, and we are onto our final issue involving the lower segments of the Blackcomb Gondola, Excalibur Gondola, and the Whistler Village Gondola.  Let's get a look at them below.

In [47]:
# get sub-table of missing lifts for three lifts we are currently interested in.
missing_lift_days(lift_df4, open_close).loc[['Blackcomb Gondola (Lower)',
                                             'Excalibur Gondola (Lower)',
                                             'Whistler Village Gondola (Lower)'], :]

,2014/2015,2015/2016,2016/2017,2017/2018,2018/2019,2019/2020,2020/2021
Blackcomb Gondola (Lower),9,22,6,6,147,107,117
Excalibur Gondola (Lower),108,151,146,152,147,106,94
Whistler Village Gondola (Lower),108,151,146,152,147,104,117


<a id='key_decision_3'></a>
First thing to note is that Blackcomb Gondola (Lower) was actually the Wizard Chair up until the 2018/2019 season (see above).  This explains why we see a low number of missing lift days up to that point.

Second, each of the 'lifts' above are subsections of larger gondola systems (Blackcomb, Excalibur, and Whistler Village Gondolas respectively).  In our experience, it is quite rare that the lower sections of the gondolas operate on their own.  When they do, it is usually when avalanche or wind conditions are severe enough that the rest of the mountain either does not open, or opens quite late.  That means the days we do have opening records for are probably quite important in helping our model identify outlier days.

So, in order for our data to be consistent, we will impute/create opening datetimes for each lift season with over 100 missing day records.  Since most of the time a gondola system opens as a whole (Gondola and Gondola (Lower) open at the same time) we will impute the records using the corresponding parent gondola opening times.  This will still preserve our data integrity because days when parent gondola and gondola (lower) times do not match exactly are going to be days of interest to our model.

In [48]:
# list to store all new opening records
new_opening_records = []

# list of gondolas needing imputation
gondolas = ['Blackcomb Gondola (Lower)', 'Excalibur Gondola (Lower)', 'Whistler Village Gondola (Lower)']

# loop over each gondola
for gondola in gondolas:
    
    # get list of indicies of records for that gondola
    gondola_indicies = lift_df4[lift_df4['lift'] == gondola].index
    
    # get list of indicies of records for the 'parent' gondola
    gondola_parent = gondola[:-8]
    gondola_parent_indicies = lift_df4[lift_df4['lift'] == gondola_parent].index
    
    # loop over the list of records for the parent gondola
    for i_parent in gondola_parent_indicies:
        
        # get 'time' field from parent
        time_parent = lift_df4.loc[i_parent, 'time']
        
        # set loop control variable to false
        entry_exists = False
        
        # loop over the list of records for the gondola to make sure an entry DNE already
        for i in gondola_indicies:
            
            #get 'time' field from gondola
            time_gondola = lift_df4.loc[i, 'time']
            
            # if y/m/d of parent gondola matches y/m/d day any record in gondola
            if time_parent.date() == time_gondola.date():

                # set loop control to True, and break out of this loop
                entry_exists = True
                break
                
        # if an entry DNE create record for gondola using 'time' of parent gondola
        if not entry_exists:
            
            # create row as a dictionary
            new_open_record = {'time': time_parent,
                               'lift': gondola,
                               'day_status': 1}
            
            # append to record list
            new_opening_records.append(new_open_record)
            
# append new opening records to our current dataFrame
lift_df5 = lift_df4.append(new_opening_records, ignore_index = True)

In [49]:
# check if it worked!

# get sub-table of missing lifts for each lift of interest.  Before, after, and parent lifts
for gondy in gondolas:
    print(f"Comparison for {gondy}")
    display(missing_lift_days(lift_df4, open_close).loc[[gondy], :]) # before imputation
    display(missing_lift_days(lift_df5, open_close).loc[[gondy], :]) # after imputation
    parent = gondy[:-8]
    display(missing_lift_days(lift_df5, open_close).loc[[parent], :]) # parent gondy - to match or less
    print()
# we expect to see the same number or fewer missing lift recrods than the parent
# for the new Lower Gondola record sets

Comparison for Blackcomb Gondola (Lower)


,2014/2015,2015/2016,2016/2017,2017/2018,2018/2019,2019/2020,2020/2021
Blackcomb Gondola (Lower),9,22,6,6,147,107,117


,2014/2015,2015/2016,2016/2017,2017/2018,2018/2019,2019/2020,2020/2021
Blackcomb Gondola (Lower),8,21,5,6,36,3,3


,2014/2015,2015/2016,2016/2017,2017/2018,2018/2019,2019/2020,2020/2021
Blackcomb Gondola,8,22,5,6,36,3,3



Comparison for Excalibur Gondola (Lower)


...

,2014/2015,2015/2016,2016/2017,2017/2018,2018/2019,2019/2020,2020/2021
Excalibur Gondola,8,23,6,6,11,2,5



Comparison for Whistler Village Gondola (Lower)


,2014/2015,2015/2016,2016/2017,2017/2018,2018/2019,2019/2020,2020/2021
Whistler Village Gondola (Lower),108,151,146,152,147,104,117


,2014/2015,2015/2016,2016/2017,2017/2018,2018/2019,2019/2020,2020/2021
Whistler Village Gondola (Lower),8,12,3,0,12,3,2


,2014/2015,2015/2016,2016/2017,2017/2018,2018/2019,2019/2020,2020/2021
Whistler Village Gondola,8,12,3,0,12,4,4


<a id='key_decision_4'></a>
From the above it looks like our imputation went well.  We can even see for a few days in later years that the lower gondolas opened when the upper gondolas didn't.  This is great information.  The only issue is that it appears this information was not tracked for 2014-2019.  It is a pity this information is missing.  This is another point that we may want to revisit if our model is looking strange.  There is a chance we will have to remove the idea that the lower portion of a gondola can open seperately from the whole.  We would do that by simply droppping the lower gondola records all together.  We will note this in the _Key Decision Points_ section and return to it if neccessary.

The (hopefully) final step in the cleaning of our lift opening data is to impute a record for each day in the season that a lift did not open.

In [50]:
# iterator function to create each day in a given season
def season_dates(opened, closed):
    """
    Takes an opening and closing date and returns each date in it in succession.
    _________________
    
    Parameters:
                opened: datetime object, the first day of a ski season
                closed: datetime object, the last day of a ski season
    ________________
    
    Yields:
           a datetime object, the next day of the ski season 
    
    """
    # get number of dates we need to generate
    days_in_season = (closed - opened).days # extract days from time delta object
    
    # loop through the number of days and yield each day; +1 to account for including the last day
    for i in range(0, days_in_season + 1):
        yield opened + datetime.timedelta(days = i)

In [51]:
def add_closed_day(lift_day, lift, closed_days):
    """
    Takes a day, lift, and list of closed days.  Creates a record of a closed day, 
    and appends it to the closed_days list
    _________________
    
    Parameters:
                lift_day: a datetime object representing the date a lift was close
                lift: a string representing the name of the lift
                closed_days: reference to a list containing all the closed days
                closed: datetime object, the last day of a ski season
    ________________
    
    Returns:
            None 
    
    """
    # create row as a dictionary
    closed_day = {'time': lift_day,
                  'lift': lift,
                  'day_status': 0}
                
    # append dictionary to list (closed_days)
    closed_days.append(closed_day)
    

In [52]:
# list to store all 'closed_days'
closed_days = []

# for each chair
for lift in lift_df5['lift'].unique():
    
    # for each season - utilize season dates stored earlier
    for opened, closed in open_close:
        
        # get smaller df for this chair and season to work with
        lift_season_df = lift_df5[(lift_df5['lift'] == lift) & 
                                  (lift_df5['time'] >= opened) & 
                                  (lift_df5['time'] <= closed)]
        
        # order by date just in case
        lift_season_df = lift_season_df.sort_values('time', ascending = True)
        
        # setup iterator to generate each season day
        season_day = season_dates(opened, closed)
        
        # loop through lift_season_df by index
        for i in lift_season_df.index:
            
            # get the next day of the season
            next_season_day = next(season_day)
            
            # while the next day of the seaon has not caught up to the next lift open date,
            # fill in the gaps with closed day records
            while next_season_day.date() < lift_season_df.loc[i, 'time'].date():
                
                add_closed_day(next_season_day, lift, closed_days)
#                 # create row as a dictionary
#                 closed_day = {'time': next_season_day,
#                               'lift': lift,
#                               'day_status': 0}
                
#                 # append dictionary to list (closed_days)
#                 closed_days.append(closed_day)
                
                # generate next date from iterator
                next_season_day = next(season_day)
            
        # in the case that the end of the season is greater than the last day the chair was open
        if lift_season_df.loc[i, 'time'].date() < closed.date():     
            
            # jump ahead one day so we don't overwrite the last open day with a closed day
            next_season_day = next(season_day)
            
            # while we haven't reached the end of the season, continue to created closed
            # days for that chair until the end of the season
            while next_season_day.date() <= closed.date():
                
                add_closed_day(next_season_day, lift, closed_days)
                
                # catch last run through iterator
                if next_season_day.date() == closed.date():
                    break
                else:
                    # generate next date from iterator
                    next_season_day = next(season_day)
                
# append closed_days to lift_df5
lift_df6 = lift_df5.append(closed_days, ignore_index = True)

In [53]:
# confirm shape before and after.
print(f"Records added: {lift_df6.shape[0] - lift_df5.shape[0]}")
print(f"Expected records to add: {missing_lift_days(lift_df5, open_close).sum().sum()}")

Records added: 4754
Expected records to add: 4754


In [54]:
# see how many missing days there are remaining for each lift.  Expecting 0!
missing_lift_days(lift_df6, open_close)

,2014/2015,2015/2016,2016/2017,2017/2018,2018/2019,2019/2020,2020/2021
Emerald,0,0,0,0,0,0,0
Excalibur Gondola,0,0,0,0,0,0,0
Excelerator,0,0,0,0,0,0,0
Blackcomb Gondola,0,0,0,0,0,0,0
Blackcomb Gondola (Lower),0,0,0,0,0,0,0
Red Chair,0,0,0,0,0,0,0
Creekside Gondola,0,0,0,0,0,0,0
Fitzsimmons,0,0,0,0,0,0,0
Olympic,0,0,0,0,0,0,0
Whistler Village Gondola,0,0,0,0,0,0,0


In [55]:
# check to see how many open days we had before and after the imputation - should match!
(lift_df5[lift_df5['day_status'] == 1]['day_status'].sum() == 
 lift_df6[lift_df6['day_status'] == 1]['day_status'].sum())

True

In [56]:
# last check to make sure we have all columns we want
lift_df6.sort_values('time')

,time,lift,day_status
22192,2014-12-31 00:00:00,Whistler Village Gondola,0
22335,2014-12-31 00:00:00,Jersey Cream,0
22388,2014-12-31 00:00:00,Crystal,0
22518,2014-12-31 00:00:00,7th Heaven,0
22724,2014-12-31 00:00:00,Glacier,0
...,...,...,...
19442,2021-03-29 09:28:27,Peak,1
19443,2021-03-29 10:09:30,Harmony,1
19444,2021-03-29 10:14:06,Excalibur Gondola,1
19445,2021-03-29 10:30:08,Symphony,1


In [57]:
# get it in date order and reset index
lift_df6 = lift_df6.sort_values('time').reset_index(drop = True)

In [58]:
# one last look
lift_df6

,time,lift,day_status
0,2014-12-31 00:00:00,Whistler Village Gondola,0
1,2014-12-31 00:00:00,Jersey Cream,0
2,2014-12-31 00:00:00,Crystal,0
3,2014-12-31 00:00:00,7th Heaven,0
4,2014-12-31 00:00:00,Glacier,0
...,...,...,...
26259,2021-03-29 09:28:27,Peak,1
26260,2021-03-29 10:09:30,Harmony,1
26261,2021-03-29 10:14:06,Excalibur Gondola,1
26262,2021-03-29 10:30:08,Symphony,1


In [59]:
# one last pull of shape for us to reference when we load the csv back up
lift_df6.shape

(26264, 3)

The final imputation above looks to have worked.  The last step is to save and export as a csv.

In [408]:
lift_df6.to_csv('./Data/PowDayHistoryClean.csv',
                index = False)

### Conclusion

We have successfully clean and prepared our target data from the PowDay push notification database for our EDA and modelling.  We can now move onto the next portion of our project.

### Key Decision Points

The purpose of this section is to categlogue any critical decisions that were made during the scope of work covered in this notebook.  It is near the head of the notebook for easy access.  If there is trouble with the model, these decision points should be revisited first.  These decisions were added as they were made and may not flow in chronological order as the document is read.
- [Key Decision 1](#key_decision_1): Dropping duplicate chair openings in a given day
- [Key Decision 2](#key_decision_2): Restricing data to season opening through 17 April each season.
- [Key Decision 3](#key_decision_3): Imputing lower gondola records with gondola records
- [Key Decision 4](#key_decision_4): Keepings lower gondola records when only 2019 onwards has any unique information compared to gondola records.
- [Key Decision 5](#key_decision_5): Keeping `Spanky's Ladder`, `Blackcomb Glacier`, and `Flute Bowl` as both things to predict and predictors for lift openings.